|  |  |  |
| ---: | :--- | :--- |
| Курс:| Введение в машинное обучение | 05 |
| Срок обучения на момент сдачи: | 2,5 месяца |
В связи с тем, что GitHub отображает не весь функционал Jupyter Notebook, просмотр доступен в nbviewer по [ссылке](https://nbviewer.jupyter.org/github/Oleg-Volontsevich/Yandex_Practicum_Projects/blob/master/project05_tariff_choosing_model/yp_project_05.ipynb).

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта 03 курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# План:  

1  [Вводные данные](#1)

*    1.1  [Описание данных](#11)
*    1.2  [Инструкция по выполнению проекта](#12)

2  [Изучение данных](#2)

3  [Подготовка данных](#3)

4  [Исследование моделей](#4)

*    4.1  [Решающее дерево](#41)
*    4.2  [Случайный лес](#42)
*    4.3  [Логистическая регрессия](#43)

5  [Проверка модели на тестовой выборке](#5)

6  [Дополнительно - проверка модели на адекватность](#6)

7  [Заключение](#7)

<a name="1"></a>
## Вводные данные

<a name="11"></a>
### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

*    сalls — количество звонков,
*    minutes — суммарная длительность звонков в минутах,
*    messages — количество sms-сообщений,
*    mb_used — израсходованный интернет-трафик в Мб,
*    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<a name="12"></a>
### Инструкция по выполнению проекта

*    Откройте и изучите данные.
*    Разделите исходные данные на обучающую, валидационную и тестовую выборки.
*    Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
*    Проверьте качество модели на тестовой выборке.
*    Дополнительное задание: проверьте модели на вменяемость.

<a name="2"></a>
## Изучение данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('users_behavior.csv')

In [3]:
data.info()
display(data.head(4))
print(data['is_ultra'].value_counts())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1


0    2229
1     985
Name: is_ultra, dtype: int64


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод по данным:**

Данные без пропусков. 

Признаки: 
* calls, 
* minutes,	
* messages, 
* mb_used. 

Целевой признак: is_ultra. (30% строк)

<a name="2"></a>
## Подготовка данных
Разделим исходные данные на обучающую, валидационную и тестовую выборки (в соотношении 60/20/20).

In [4]:
data_train, data_testvalid = train_test_split(data, test_size=0.4, random_state=123)
data_test, data_valid = train_test_split(data_testvalid, test_size=0.5, random_state=123)

In [5]:
display(data_train.describe())
display(data_valid.describe())
display(data_test.describe()) 

,calls,minutes,messages,mb_used,is_ultra
count,1928.000000,1928.000000,1928.000000,1928.000000,1928.000000
mean,63.117220,439.033734,38.156120,17203.078242,0.308091
std,33.439266,236.342509,36.684478,7556.799702,0.461824
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,276.607500,9.000000,12472.822500,0.000000
50%,62.000000,430.090000,29.000000,16849.685000,0.000000
75%,82.000000,578.700000,56.000000,21326.615000,1.000000
max,203.000000,1566.450000,224.000000,48351.610000,1.000000


,calls,minutes,messages,mb_used,is_ultra
count,643.000000,643.000000,643.000000,643.000000,643.000000
mean,63.570762,442.875801,37.388802,17156.622877,0.306376
std,32.937388,230.857527,34.698990,7428.841067,0.461347
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,281.270000,8.000000,12706.160000,0.000000
50%,63.000000,447.070000,30.000000,16917.440000,0.000000
75%,82.500000,571.150000,58.500000,21584.375000,1.000000
max,244.000000,1632.060000,188.000000,43931.390000,1.000000


,calls,minutes,messages,mb_used,is_ultra
count,643.000000,643.000000,643.000000,643.000000,643.000000
mean,62.272162,431.068212,39.548989,17272.504432,0.301711
std,32.960360,233.113206,35.969293,7763.148353,0.459358
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,38.000000,263.295000,10.000000,12453.180000,0.000000
50%,60.000000,419.720000,31.000000,17275.470000,0.000000
75%,83.000000,557.580000,57.500000,21515.760000,1.000000
max,189.000000,1364.830000,201.000000,49745.730000,1.000000


Целевой признак в обучающую, валидационную и тестовую выборки распределился равномерно.

In [6]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

<a name="4"></a>
## Исследование моделей
Исследуем качество трех известных нам данный момент моделей: DecisionTreeClassifier, RandomForestClassifier, LogisticRegression.

<a name="41"></a>
### Решающее дерево

In [7]:
best_DecisionTree_model = None
best_DecisionTree_result = 0
best_DecisionTree_depth = 0

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_DecisionTree_result:
        best_DecisionTree_model = model
        best_DecisionTree_result = result
        best_DecisionTree_depth = depth

print('Accuracy лучшего дерева: {:.3f}'.format(best_DecisionTree_result), 'для max_depth =', best_DecisionTree_depth)

Accuracy лучшего дерева: 0.796 для max_depth = 3


<a name="s3_2"></a> 
### Случайный лес

In [8]:
best_RandomForest_model = None
best_RandomForest_result = 0
best_est = 0
best_RandomForest_depth = 0

for est in range(1, 17):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid)
        if result > best_RandomForest_result:
            best_RandomForest_model = model
            best_est = est
            best_RandomForest_result = result
            best_RandomForest_depth = depth

print('Accuracy лучшего леса: {:.3f}'.format(best_RandomForest_result), 
      'для леса из', best_est, 'деревьев глубиной', best_RandomForest_depth)

Accuracy лучшего леса: 0.806 для леса из 6 деревьев глубиной 4


<a name="s3_3"></a> 
### Логистическая регрессия

In [9]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)

print('Accuracy модели логистической регрессии: {:.3f}'.format(result))

Accuracy модели логистической регрессии: 0.697


**Вывод по моделям:**

Хуже всего на валидационной выборке себя показала логистическая регрессия (69,7%). Результаты для решающего дерева и случайного леса практически одинаковые (79,6% и 80,6% соответственно). Разница в точности в 1% можно считать значительной. Если нет потребности в скорости предсказания - то в качестве модели лучше выбрать случайный лес из 6 деревьев глубиной 4.

<a name="5"></a>
## Проверка модели на тестовой выборке

In [10]:
test_result = best_RandomForest_model.score(features_test, target_test)
print('Точность на тестовой выборке: {:.3f}'.format(test_result))

Точность на тестовой выборке: 0.804


Требуемый порог по точности преодолели.

<a name="6"></a>
## Дополнительно - проверка модели на адекватность

Очевидно, что заполнив предсказание нулями получим точность 70%. 
Первым делом была мысль попробовать создать предсказание по собственному алгоритму (например заполнить единицами 30% наиболее активных абонентов) и посмотреть результат, но не уверен что это то, что подразумевалось в этом задании.  
Попробуем псевдопредсказатель DummyClassifier. 
  'most_frequent' - заполнит предсказание наиболее распространенным значением 
  'stratified' - заполнит значениями с сохранением доли в тестовой выборке
  'uniform' - заполнит значениями в равной пропорции

In [11]:
strategies = ['most_frequent', 'stratified', 'uniform']
for strat in strategies:
    model = DummyClassifier(strategy = strat, random_state = 123) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    print('Accuracy DummyClassifier: {:.3f}'.format(result), 'для стратегии', strat)

Accuracy DummyClassifier: 0.694 для стратегии most_frequent
Accuracy DummyClassifier: 0.585 для стратегии stratified
Accuracy DummyClassifier: 0.496 для стратегии uniform


<a name="7"></a>
## Заключение

Модель случайного дерева дает точность более 80,4%. Это на 10% лучше чем во всех случаях предсказывать наиболее популярный тариф. Для поставленной задачи такой точности достаточно, но для более серьезных целей необходимо увеличивать точность.

Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75